In [1]:
import pandas as pd
fn = '../src/data/raw/Trees_data.csv'
tree_df = pd.read_csv(fn)
# tree_df.sum()

In [114]:
tree_df.head()

,the_geom,OBJECTID,ID,UniqueID,Address or block number,Interpolated address,Street,On street or park,From street,To street,...,Rating,Tree trimming area,Subzone,Inventory date,Inventory time,Inspection date,Inspection time,Notes,Active,Genus
0,POINT (-87.686657011238 42.04962539344),90570,22108,JB 20140520110056,1007,NaN,CHURCH ST,OAK AVE,ALLEY,ALLEY,...,9,NaN,4A,05/20/2014 12:00:00 AM +0000,NaN,05/20/2014 12:00:00 AM +0000,11:00:56,NaN,1,Gleditsia
1,POINT (-87.724424728412 42.065557689441),84188,15709,BiB 20140530085326,3235,NaN,HARTZELL ST,HARTZELL ST,ALLEY,ALLEY,...,5,NaN,1A,05/30/2014 12:00:00 AM +0000,NaN,05/30/2014 12:00:00 AM +0000,08:53:26,NaN,1,Celtis
2,POINT (-87.683002482292 42.032848194212),72384,3841,AR1 20140507160655,813,NaN,WASHINGTON ST,WASHINGTON ST,ALLEY,ALLEY,...,4,NaN,6A,05/07/2014 12:00:00 AM +0000,NaN,05/07/2014 12:00:00 AM +0000,16:11:34,NaN,1,Gymnocladus
3,POINT (-87.69493760295 42.053841563643),86663,18196,GB 20140516144031,1630,NaN,FOSTER ST,FOSTER ST,ALLEY,ALLEY,...,6,NaN,2D,05/16/2014 12:00:00 AM +0000,NaN,05/16/2014 12:00:00 AM +0000,14:40:31,NaN,1,Celtis
4,POINT (-87.708236901829 42.049774590172),80581,12092,BB 20140521110309,2404,X,BECK,BECK,ALLEY,ALLEY,...,5,NaN,3A,06/04/2014 12:00:00 AM +0000,NaN,06/04/2014 12:00:00 AM +0000,20:17:55,NaN,1,Acer


In [145]:
dead_boo = (
    (tree_df.Condition == 'Poor') |
    (tree_df.Condition == 'nan') |
    (tree_df.Condition == 'Dead') |
    (tree_df.Condition == 'Critical') |
    (tree_df.Active == 0) |
    (tree_df.Species == 'stump') |
    (tree_df.Species == 'Vacant')
)
alive_df = tree_df[~dead_boo]
len(alive_df)

30021

In [4]:
genus_list = pd.Series([blah[0] for blah in tree_df['Species'].str.split()]).unique()
tree_df['Genus'] = [blah[0] for blah in tree_df['Species'].str.split()]
tree_df['Genus'].unique()

array(['Gleditsia', 'Celtis', 'Gymnocladus', 'Acer', 'Fraxinus', 'Picea',
       'Populus', 'Aesculus', 'Pyrus', 'Ulmus', 'Vacant', 'Ginkgo',
       'Quercus', 'Tilia', 'Crataegus', 'Alnus', 'Syringa', 'unknown',
       'Platanus', 'stump', 'Pinus', 'Rhamnus', 'Thuja', 'Ostrya',
       'Morus', 'Cercis', 'Prunus', 'Hamamelis', 'Malus', 'Betula',
       'Cercidiphyllum', 'Metasequoia', 'Amelanchier', 'Liriodendron',
       'Catalpa', 'Liquidambar', 'Cornus', 'Carpinus', 'Styphnolobium',
       'Salix', 'Phellodendron', 'Nyssa', 'Pseudotsuga', 'Ligustrum',
       'Magnolia', 'Koelreuteria', 'Fagus', 'Juniperus', 'Cotinus',
       'Corylus', 'Juglans', 'Tsuga', 'Zelkova', 'Carya', 'Taxodium',
       'Parrotia', 'Robinia', 'Cladrastis', 'Ailanthus', 'Rhus', 'Larix',
       'Eucommia', 'Viburnum', 'Abies', 'Taxus', 'Lonicera', 'Hibiscus',
       'Maackia', 'Ilex', 'Halesia', 'Paulownia', 'Chionanthus', 'Sorbus',
       'Albizia'], dtype=object)

In [61]:
map_dict = {
    "Gleditsia":"Honeylocust",
    #"Celtis":"Hackberry",
    #"Gymnocladus":"Other", #Kentucky Coffeetree
    "Acer":"Maple",
    "Fraxinus":"Ash",
    "Quercus":"Oak",
    "Tilia":"Linden",
    "Ulmus":"Elm",
    #"Populus":"Poplar"
}

In [ ]:
tree_df.iloc[0:2792,:].to_csv('Trees_genus.csv', index=False)

In [144]:
#gb = tree_df.groupby(list(tree_df['Genus'].values))
#tree_df['idx'] = tree_df.groupby('Genus').sum()
# keep_list = ['Inventory date', 'Active']
# tree_df.pivot(index='Genus',columns='idx')[keep_list]
genus_sum = alive_df.groupby('Genus').sum()
genus_sum.sort_values(by=['DBH'], axis=0, ascending=False, inplace=True)
genus_sum = genus_sum.div(sum(genus_sum.DBH))*100
genus_sum = genus_sum['DBH']
genus_sum.index = genus_sum.index.map(map_dict).fillna('Other')
ev_canopy = genus_sum.groupby('Genus').sum().sort_values(ascending=False)
ev_canopy = ev_canopy.round().astype(int)
ev_canopy.name = 'evanston_canopy'
#regional canopy from Chicago Regional Tree Initiative Evanston Report (in /references/Evanston.PDF), page 9
#'Linden' from US Forest Reserve 'Urban trees and forests of the Chicago region' (in/references/rb_nrs84.pdf), page 40
reg_canopy = pd.Series(
    [32, 24, 16, 12, 7, 7, 2], 
    index=['Maple','Other','Ash','Honeylocust','Oak','Elm','Linden'],
    name = 'regional_canopy'
)
common_canopy = pd.concat([ev_canopy,reg_canopy], axis = 1, sort=False)
ev_other = common_canopy['evanston_canopy'].values[0]
common_canopy

,evanston_canopy,regional_canopy
Other,26,24
Elm,23,7
Maple,21,32
Honeylocust,12,12
Linden,10,2
Oak,7,7
Ash,1,16


In [147]:
common_canopy.to_csv('../src/data/interim/Common_canopy.csv')